## Import Packages

In [1]:
import config
import utils

import json
import pickle
import os
import numpy as np
from tqdm.notebook import tqdm

## Make directory for saving

In [2]:
os.makedirs(config.skeleton_pkl_dir, exist_ok=True)
os.makedirs(config.skeleton_npz_dir, exist_ok=True)

## Start processing

In [3]:
video_idx = [v_idx for v_idx in os.listdir(config.mocap_dir) if v_idx[0]!='.']
for v_idx in video_idx:
    result_dir = os.path.join(config.mocap_dir, v_idx)
    bbox_dir = os.path.join(result_dir, 'bbox')
    mocap_dir = os.path.join(result_dir, 'mocap')
    
    pkl_dir = os.path.join(config.skeleton_pkl_dir, v_idx)
    npz_dir = os.path.join(config.skeleton_npz_dir, v_idx)
    
    os.makedirs(pkl_dir, exist_ok=True)
    os.makedirs(npz_dir, exist_ok=True)
    
    for mocap_fp in tqdm(sorted(os.listdir(mocap_dir))):
        scene_name = mocap_fp.split('/')[-1][:14]
        
        bbox_fp = os.path.join(bbox_dir, scene_name+'_bbox.json')
        mocap_fp = os.path.join(mocap_dir, scene_name+'_prediction_result.pkl')
        
        ### load info
        bbox = json.load(open(bbox_fp, 'rb'))
        mocap = pickle.load(open(mocap_fp, 'rb'))
        
        ### get bbox size
        bbox_np = np.array(bbox['body_bbox_list'])
        bbox_size = [bb[2]*bb[3] for bb in bbox_np]

        ### only choose the biggest bbox
        max_idx = np.argmax(bbox_size)
        curr_body_bbox = bbox_np[max_idx]
        curr_mocap = mocap['pred_output_list'][max_idx]

        ### extract body and hand pose
        curr_body_pose = curr_mocap['pred_joints_img']
        curr_open_pose = curr_body_pose[:25]
        curr_aux_pose = curr_body_pose[25:]

        curr_right_hand = curr_mocap['right_hand_joints_img_coord']
        curr_left_hand = curr_mocap['left_hand_joints_img_coord']
        
        # extract hand bbox
        if None not in mocap['hand_bbox_list']:
            curr_left_hand_bbox = np.asarray(mocap['hand_bbox_list'][max_idx]['left_hand'])
            curr_right_hand_bbox = np.asarray(mocap['hand_bbox_list'][max_idx]['right_hand'])
            if curr_left_hand_bbox.all() == None:
                curr_left_hand_bbox = np.array([0, 0, 0, 0])
            if curr_right_hand_bbox.all() == None:
                curr_right_hand_bbox = np.array([0, 0, 0, 0])

            
        ### extract simplified pose
        curr_simple_pose = curr_open_pose[:15] # 15 by 3

        ### process for .npz file.
        left_hand_info, left_hand_array = utils.get_hand_info(curr_left_hand)
        right_hand_info, right_hand_array = utils.get_hand_info(curr_right_hand)

        left_heel_info, right_heel_info, heel_array = utils.get_heel_info(curr_open_pose)

        head_info, head_array = utils.get_head_info(curr_open_pose)

        ### prepare dictionary
        pkl_saver = dict()
        pkl_saver['open_pose'] = curr_open_pose
        pkl_saver['aux_pose'] = curr_aux_pose
        pkl_saver['simple_pose'] = curr_simple_pose
        pkl_saver['left_hand'] = curr_left_hand
        pkl_saver['right_hand'] = curr_right_hand
        pkl_saver['hand_info'] = {'left':left_hand_info, 'right':right_hand_info}
        pkl_saver['heel_info'] = {'left':left_heel_info, 'right':right_heel_info}
        pkl_saver['head_info'] = head_info
        pkl_saver['body_bbox'] = curr_body_bbox
        pkl_saver['left_hand_bbox'] = curr_left_hand_bbox
        pkl_saver['right_hand_bbox'] = curr_right_hand_bbox

        ### prepare the array
        npz_saver = [curr_simple_pose.flatten(), head_array, left_hand_array, right_hand_array, heel_array, 
                     curr_body_bbox, curr_left_hand_bbox, curr_right_hand_bbox]
        npz_saver = np.concatenate(npz_saver)
        # 0  ~ 45 : pose that was 15 by 3
        # 45 ~ 51 : head pos(3), vec(3)
        # 51 ~ 63 : left hand thumb vector(3), index vector(3), palm vector(3), wrist pos(3)
        # 63 ~ 75 : right hand thumb vector(3), index vector(3), palm vector(3), wrist pos(3)
        # 75 ~ 87 : left heel pos (3), left foot vec (3), right heel pos (3), right foot vec (3)
        # 87 ~ 91 : body bbox (4) left hand bbox (4) right hand bbox(4)

        ### save the data
        pkl_fp = os.path.join(pkl_dir, scene_name+'.pkl')
        npz_fp = os.path.join(npz_dir, scene_name+'.npz')
        pickle.dump(pkl_saver, open(pkl_fp, 'wb'))
        np.savez(npz_fp, npz_saver)